In [2]:
!pip install torch_snippets
import torch
from torch_snippets import *
from torchvision import transforms as T
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from torchvision import transforms as T
preprocess = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    T.Lambda(lambda x: x.mul_(255))
])

postprocess=T.Compose([
    T.Lambda(lambda x: x.mul_(1./255)),
    T.Normalize(
        mean=[-0.485/0.299, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.255])
])

In [5]:
#@ Gram Matrix module:
class GramMatrix(torch.nn.Module):
  def forward(self, input):
    b, c, h, w=input.size()
    feature=input.view(b, c, h*w)
    G= feature @ feature.transpose(1, 2)
    G.div_(h*w)
    return G

In [6]:
class GramMSELoss(torch.nn.Module):
  def forward(self, input, target):
    output=F.mse_loss(GramMatrix()(input), target)
    return(out)

In [13]:
#@ VGG-19 modified:
class vgg19_modified(torch.nn.Module):
  def __init__(self):
    super().__init__()

    #extracting the feature:
    features=list(models.vgg19(pretrained=True).features)
    self.features=torch.nn.ModuleList(features).eval()

  def forward(self, x, layers=[]):
    order=np.argsort(layers)
    _results, results=[], []
    for ix, model in enumerate(self.features):
      x=model(x)
      if ix in layers: _results.append(x)
    for o in order: results.append(_results[o])
    return results if layers is not [] else x

In [14]:
from torchvision import models
vgg=vgg19_modified().to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 91.8MB/s]


In [15]:
#@ Importing content and style image:
!wget https://www.dropbox.com/s/z1y0fy2r6z6m6py/60.jpg
!wget https://www.dropbox.com/s/1svdliljyo0a98v/style_image.png

--2025-03-08 15:19:29--  https://www.dropbox.com/s/z1y0fy2r6z6m6py/60.jpg
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/1konv3e8jq9ng48mse478/60.jpg?rlkey=nsxg1iud7ie8tpeos1aclfxkt [following]
--2025-03-08 15:19:29--  https://www.dropbox.com/scl/fi/1konv3e8jq9ng48mse478/60.jpg?rlkey=nsxg1iud7ie8tpeos1aclfxkt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘60.jpg’

60.jpg                  [ <=>                ] 156.78K  --.-KB/s    in 0.04s   

2025-03-08 15:19:30 (3.95 MB/s) - ‘60.jpg’ saved [160546]

--2025-03-08 15:19:30--  https://www.dropbox.com/s/1svdliljyo0a98v/style_image.png
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6021:18::a27d:4112
Connecting t

In [24]:
imgs=[Image.open(path).resize((512, 512)).convert('RGB') for path in ['style_image.png', '60.jpg']]
style_image, content_image = [preprocess(img).to(device).unsqueeze(0) for img in imgs]


UnidentifiedImageError: cannot identify image file 'style_image.png'

In [23]:
opt_img=content_image.data.clone()
opt_img.requires_grad=True

NameError: name 'content_image' is not defined

In [ ]:
style_layers=[0, 5, 10, 19, 28]
content_layers=[21]
loss_layers=style_layers + content_layers

In [ ]:
loss_fns=[GramMSELoss()] * len(style_layers) + [nn.MSELoss()] * len(content_layers)
loss_fns=[loss_fn.to(device) for loss_fn in loss_fns]

In [ ]:
#@ defining weightage associated with content and style loss:
style_weights=[1000/n**2 for n in [64, 128, 256, 512, 512]]
content_weights=[1]
weights=style_weights + content_weights

In [ ]:
style_targets=[GramMatrix()(A).detach() for A in vgg(style_image, style_layers)]
content_targets=[A.detach() for A in vgg(content_image, content_layers)]
targets=style_targets + content_layers

In [ ]:
import matplotlib.pyplot as plt

max_iters = 500
optimizer = optim.LBFGS([opt_img])

loss_values = []  # Store loss values

iters = 0
while iters < max_iters:
    def closure():
        global iters
        iters += 1
        optimizer.zero_grad()
        out = vgg(opt_img, loss_layers)
        layer_losses = [weights[a] * loss_fns[a](A, targets[a]) for a, A in enumerate(out)]
        loss = sum(layer_losses)
        loss.backward()
        loss_values.append(loss.item())  # Store loss
        return loss

    optimizer.step(closure)

# Plot the loss variation
plt.plot(loss_values)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Loss Variation During Optimization")
plt.show()


In [ ]:
With torch.no_grad()
 out_img = postprocess(opt_img[0]).permute(1,2,0)
show(out_img)